In [293]:
%load_ext autoreload
%autoreload 2
from utils import *
from IPython.display import display, HTML
ins = BrainSpan()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Variants

In [74]:
nature13908_vars = "/Users/jiayao/Work/BrainDisorders/data/nature13908-s2/Supplementary_Table_2.xlsx"
all_var = pd.read_excel(nature13908_vars)
display(all_var.head(2))
LGD = set(["noStart", "frame-shift", "nonsense", "noEnd", "no-frame-shift-newStop","splice-site"])
MIS = set(["missense"])
SYN = set(["synonymous"])
InPro = set(["pMsM", "pMsF", "pFsF", "pF", "pM", "pFsM"])
InSib = set(["pMsM", "pMsF", "pFsF", "sF", "sM", "pFsM"])

,familyId,location,variant,vcfVariant,inChild,fromParent,effectGene,effectType,familyDescription,CSHL,YALE,UW,IossifovWE2012,EichlerWE2012,StateWE2012,EichlerTG2012
0,14505,1:897349,sub(G->A),1:897349:G:A,pM,NaN,KLHL17,synonymous,pMsM,NaN,NaN,weak:valid,NaN,NaN,NaN,NaN
1,14320,3:127395222,sub(T->G),3:127395222:T:G,pF,NaN,ABTB1,noStart,pFsM,strong,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
DatDir = "/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/"
ProLGD = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(LGD))]
ProMIS = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(MIS))]
ProSYN = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(SYN))]
SibLGD = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(LGD))]
SibMIS = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(MIS))]
SibSYN = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(SYN))]
SibNSYN = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(MIS.union(LGD)))]
ProLGD.to_excel(DatDir + "wigler2014ProLGD.xlsx", index=False)
ProMIS.to_excel(DatDir + "wigler2014ProMIS.xlsx", index=False)
ProSYN.to_excel(DatDir + "wigler2014ProSYN.xlsx", index=False)
SibLGD.to_excel(DatDir + "wigler2014SibLGD.xlsx", index=False)
SibMIS.to_excel(DatDir + "wigler2014SibMIS.xlsx", index=False)
SibSYN.to_excel(DatDir + "wigler2014SibSYN.xlsx", index=False)
SibNSYN.to_excel(DatDir + "wigler2014SibNSYN.xlsx", index=False)

In [76]:
DatDir = "/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/"
ProLGD = pd.read_excel(DatDir+"wigler2014SibSYN.xlsx")
print ProLGD.shape, len(set(ProLGD["effectGene"].values))

(489, 16) 468


In [25]:
GeneCount = ProLGD.groupby("effectGene")["effectGene"].count()
ProLGD.insert(list(ProLGD.columns.values).index("effectGene")+1, "GeneCount", ProLGD.apply
                 (lambda row: GeneCount[row["effectGene"]], axis=1), allow_duplicates=False )
ProLGD.head()
RecProLGD = ProLGD[ProLGD["GeneCount"]>=2]

In [26]:
RecProLGD.head()

,familyId,location,variant,vcfVariant,inChild,fromParent,effectGene,GeneCount,effectType,familyDescription,CSHL,YALE,UW,IossifovWE2012,EichlerWE2012,StateWE2012,EichlerTG2012
108,13545,20:49509095,ins(T),20:49509094:G:GT,pM,NaN,ADNP,2,frame-shift,pM,NaN,strong:valid,NaN,NaN,NaN,NaN,yes
123,12130,20:49510028,del(2),20:49510027:CTT:C,pF,NaN,ADNP,2,frame-shift,pF,NaN,NaN,strong:valid,NaN,yes,NaN,NaN
242,12645,4:114232545,sub(C->T),4:114232545:C:T,pM,NaN,ANK2,3,nonsense,pMsM,strong:valid,NaN,NaN,yes,NaN,NaN,NaN
259,14256,4:114251469,sub(C->T),4:114251469:C:T,pM,NaN,ANK2,3,nonsense,pMsF,NaN,strong:valid,NaN,NaN,NaN,NaN,NaN
264,13768,4:114277600,del(1),4:114277599:AG:A,pM,NaN,ANK2,3,frame-shift,pMsF,strong,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
print RecProLGD.shape

(67, 17)


In [28]:
RecProLGD.to_excel(DatDir + "wigler2014RecProLGD.xlsx", index=False)

# Go Term

In [43]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()

from goatools.associations import read_ncbi_gene2go

go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], go2geneids=True)
print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go2geneids_human)))

from goatools.go_search import GoSearch

srchhelp = GoSearch("go-basic.obo", go2items=go2geneids_human)

  wget.download(http://purl.obolibrary.org/obo/go/go-basic.obo out=go-basic.obo)
  wget.download(ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz out=gene2go.gz)


  gunzip gene2go.gz
  17,983 items READ: gene2go
17983 GO terms associated with human NCBI Entrez GeneIDs
go-basic.obo: fmt(1.2) rel(2018-10-23) 47,358 GO Terms; optional_attrs(comment def relationship synonym xref)


In [30]:
from goatools import obo_parser
import wget
import os
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = os.getcwd() + '/data'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.mkdir(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

def find_children(term1, go, go_term_set={}, ret=False):
    for term2 in term1.children:
        go_term_set.update({term2})
        
        # Recurse on term to find all children
        find_children(term2, go, go_term_set)
    if(ret):
        return go_term_set
    
# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'
go = obo_parser.GODag(go_obo)

/Users/jiayao/Work/BrainDisorders/src/data/go-basic.obo: fmt(1.2) rel(2018-10-19) 47,359 GO Terms


In [100]:
def GetALLGo(GoID):
    Root = go[GoID]
    all_go = Root.get_all_children()
    all_go.add(GoID)
    return all_go

In [101]:
chromatin_organization = "GO:0006325"
synaptic_signaling = "GO:0099536"
synapse = "GO:0045202"
cortical_cytoskeleton = "GO:0030863"
presynaptic_cytoskeleton = "GO:0099569"
postsynaptic_cytoskeleton = "GO:0099571"
channel_activity = "GO:0015267"

In [102]:
goset_chromatin_organization = GetALLGo(chromatin_organization)
goset_synaptic_signaling = GetALLGo(synaptic_signaling)
goset_synapse = GetALLGo(synapse)
goset_presynaptic_cytoskeleton = GetALLGo(presynaptic_cytoskeleton)
goset_postsynaptic_cytoskeleton = GetALLGo(postsynaptic_cytoskeleton)
goset_cortical_cytoskeleton = GetALLGo(cortical_cytoskeleton)
goset_channel_activity = GetALLGo(channel_activity)

In [103]:
chromatin_geneids = srchhelp.get_items(goset_chromatin_organization)
synaptic_signaling_geneids = srchhelp.get_items(goset_synaptic_signaling)
synapse_geneids = srchhelp.get_items(goset_synapse)
presynaptic_cyto_geneids = srchhelp.get_items(goset_presynaptic_cytoskeleton)
postsynaptic_cyto_geneids = srchhelp.get_items(goset_postsynaptic_cytoskeleton)
cortical_cytoskeleton_geneids = srchhelp.get_items(goset_cortical_cytoskeleton)
channel_geneids = srchhelp.get_items(goset_channel_activity)

In [104]:
print len(chromatin_geneids)
print len(synaptic_signaling_geneids)
print len(synapse_geneids)
print len(presynaptic_cyto_geneids)
print len(postsynaptic_cyto_geneids)
print len(cortical_cytoskeleton_geneids)
print len(channel_geneids)

713
324
714
8
7
74
467


In [105]:
cyto_sig = presynaptic_cyto_geneids.union(postsynaptic_cyto_geneids).union(cortical_cytoskeleton_geneids).union(synaptic_signaling_geneids)
chromatin = chromatin_geneids
synapse = synapse_geneids
channel = channel_geneids

In [106]:
dat_dir = '/Users/jiayao/Work/BrainDisorders/data/functions/'
def to_file(filename, _list):
    fout = open(filename, 'wb')
    for term in _list:
        fout.write("{}\n".format(term))
#print chromatin_geneids

to_file(dat_dir+"go_cyto_sig.list", cyto_sig)
to_file(dat_dir+"go_chromatin.list", chromatin)
to_file(dat_dir+"go_synapse.list", synapse)
to_file(dat_dir+"go_channel.list", channel)

# Select Recurrent Mutation Data

In [305]:
BS_row_meta = "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/rows_metadata.csv"
BS_row_meta_withvar = "/Users/jiayao/Work/BrainDisorders/src/data/RecGenesExonsTargetedByLGD_wilger2014_predicted.xlsx"
wigler_fam_info = "/Users/jiayao/Work/BrainDisorders/data/nature13908-s2/Supplementary_Table_1.xlsx"
VarFil = "/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/wigler2014RecProLGD.xlsx"
df_row = pd.read_excel(BS_row_meta_withvar, index_col="row_num")
df_row = pd.read_csv(BS_row_meta, index_col="row_num")
df_fam = pd.read_excel(wigler_fam_info)
df_var = pd.read_excel(VarFil)

# Columns Meta Data
bp_exon_col_meta = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/columns_metadata.csv", delimiter=",")
bp_exon_col_meta.age.unique()
bp_exon_col_meta["Period"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[0], axis=1)
bp_exon_col_meta["Stage"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[1], axis=1)
bp_exon_col_meta["Description"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[2], axis=1)
# Exon RPKM Norm by Gene RPKM
NormExonExp = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/NormedExonExpMatrix.csv", header = None)
NormExonExp.head(2)
# Exon RPKM Matrix
ExonExp = pd.read_csv("../data/expression/brainspan/exons_matrix/expression_matrix.csv", header=None)
display(ExonExp.head(2))
Regionsgt20 = ['OFC', 'VFC', 'HIP', 'ITC', 'AMY', 'DFC', 'STC', 'MFC', 'STR', 'IPC', 
            'V1C', 'S1C', 'A1C', 'M1C', 'CBC', 'MD']
RegionsCortex = ["OFC", "VFC", "ITC", "DFC", "STC", "MFC"]
bp_exon_row_meta = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/rows_metadata.csv", delimiter=",")

,0,1,2,3,4,5,6,7,8,9,...,515,516,517,518,519,520,521,522,523,524
0,1,13.636583,40.862901,32.178981,27.939465,29.324584,23.885710,24.887880,16.432391,45.570680,...,4.347437,4.101772,2.987192,3.862508,3.362597,2.566516,3.049593,5.025746,1.893471,8.942586
1,2,31.731240,86.587706,60.131360,57.718046,58.326412,54.090394,50.138146,35.408905,94.937737,...,6.332389,5.479924,5.946744,6.017439,5.585987,3.870925,4.083335,7.451925,2.946266,3.868906


In [167]:
ins = BrainSpan()
Genes = list(set(df_var["effectGene"].values))
rows = df_row[df_row["gene_symbol"].isin(Genes)]
idxs = rows.index.values
matrix = ExonExp[ExonExp[0].isin(idxs)]

NameError: name 'df_var' is not defined

In [107]:
stage2column_num = {}
for stage in ins.Stages:
    column_num = set(bp_exon_col_meta[bp_exon_col_meta["Period"]==stage]["column_num"].values)
    stage2column_num[stage] = column_num

In [144]:
OutFil = "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/exon_ralexp_avg2time.csv"
writer = csv.writer(open(OutFil, 'wb'))
header = ["column_num"] + ["mean_"+ x for x in ins.Stages] + ["var_mean_"+ x for x in ins.Stages]
writer.writerow(header)
matrix = NormExonExp
for i, row in matrix.iterrows():
    means, var_means = [], []
    for stage in ins.Stages:
        points = np.array([])
        for sample in stage2column_num[stage]:
            exp = row[sample]
            points = np.append(points, exp)
        mean, var_mean = round(points.mean(),6), round(points.var() / points.size, 6)
        means.append(mean)
        var_means.append(var_mean)
    writer.writerow([int(row[0])]+means+var_means)

In [168]:
GeneExp = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/qn.expression_matrix.csv", header=None)
GeneRow = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/rows_metadata.csv")
GeneCol = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/columns_metadata.csv")
GeneCol["Period"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[0], axis=1)
GeneCol["Stage"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[1], axis=1)
GeneCol["Description"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[2], axis=1)
GeneDat = [GeneExp, GeneRow, GeneCol]
gene_stage2column_num = {}
for stage in ins.Stages:
    column_num = set(GeneCol[GeneCol["Period"]==stage]["column_num"].values)
    gene_stage2column_num[stage] = column_num

In [149]:
OutFil = "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/gene_exp_avg2time.csv"
writer = csv.writer(open(OutFil, 'wb'))
header = ["column_num"] + ["mean_"+ x for x in ins.Stages] + ["var_mean_"+ x for x in ins.Stages]
writer.writerow(header)
matrix = GeneExp
for i, row in matrix.iterrows():
    means, var_means = [], []
    for stage in ins.Stages:
        points = np.array([])
        for sample in gene_stage2column_num[stage]:
            exp = row[sample]
            points = np.append(points, exp)
        mean, var_mean = round(points.mean(),6), round(points.var() / points.size, 6)
        means.append(mean)
        var_means.append(var_mean)
    writer.writerow([int(row[0])]+means+var_means)

In [308]:
qnExonExp = pd.read_csv("../data/expression/brainspan/exons_matrix/qn_exons_matrix.csv", header=None)

In [108]:
OutFil = "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/qn.exon_exp_avg2time.csv"
writer = csv.writer(open(OutFil, 'wb'))
header = ["column_num"] + ["mean_"+ x for x in ins.Stages] + ["var_mean_"+ x for x in ins.Stages]
writer.writerow(header)
matrix = ExonExp
for i, row in matrix.iterrows():
    means, var_means = [], []
    for stage in ins.Stages:
        points = np.array([])
        for sample in stage2column_num[stage]:
            exp = row[sample]
            points = np.append(points, exp)
        mean, var_mean = round(points.mean(),6), round(points.var() / points.size, 6)
        means.append(mean)
        var_means.append(var_mean)
    writer.writerow([int(row[0])]+means+var_means)

In [165]:
exon_exp_avg2time = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/exon_exp_avg2time.csv")

In [167]:
OutFil = "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/exon_selfnorm_avg2time.csv"
writer = csv.writer(open(OutFil, 'wb'))
header = ["column_num"] + ["mean_total_time"] + [x for x in ins.Stages]
writer.writerow(header)
matrix = exon_exp_avg2time
for i, row in matrix.iterrows():
    dat = row[1:12].values
    mean = np.mean(dat)
    if mean != 0:
        tmp = [float(x)/mean for x in dat]
    else:
        tmp = [0] * 11
    writer.writerow([int(row[0])]+[mean]+tmp)

In [33]:
# CDS
f_cds = open("/Users/jiayao/Work/Resources/gencode.v10.cds.gtf", 'rb')
#f_cds = open("/Users/jiayao/Work/Resources/test", 'rb')
F = f_cds.readlines()
res = {}
for i,l in enumerate(F):
    llist = l.strip().split("\t")
    if llist[2] == "CDS":    
        info = llist[8].split(";")
        gene = info[4].split()[-1].strip('"')
        if gene not in res:
            res[gene] = {}
        cds_s = int(llist[3])
        cds_e = int(llist[4])
        e_line = F[i-1]
        e_llist = e_line.strip().split("\t")
        exon_s, exon_e = int(e_llist[3]), int(e_llist[4])
        if exon_s not in res[gene]:
        #print exon_s <= cds_s, exon_e >= cds_e 
            res[gene][exon_s] = (cds_s, cds_e)

In [34]:
fout = open("cds.dict", 'wb')
for k,v in res.items():
    for kk, vv in v.items():
        fout.write("{}\t{}\t{}\t{}\n".format(k, kk, vv[0], vv[1]))

In [50]:
df = pd.DataFrame({{'A': 5, 'B': 2, 'C': 3, 'D': 4},
                   {'A': 4, 'B': 1, 'C': 4, 'D': 2},
                   {'A': 3, 'B': 4, 'C': 6, 'D': 8}})

TypeError: unhashable type: 'dict'

In [51]:
df

,C1,C2,C3
A,5,4,3
B,2,1,4
C,3,4,6
D,4,2,8


In [48]:
df_n = quantileNormalize(df)

In [49]:
df_n

,C1,C2,C3
A,5.666667,4.666667,2.000000
B,2.000000,2.000000,3.000000
C,3.000000,4.666667,4.666667
D,4.666667,3.000000,5.666667


In [74]:
ExonExp = pd.read_csv("../data/expression/brainspan/exons_matrix/expression_matrix.csv", header=None, index_col=0)
display(ExonExp.head(2))

,1,2,3,4,5,6,7,8,9,10,...,515,516,517,518,519,520,521,522,523,524
0,,,,,,,,,,,,,,,,,,,,,
1,13.636583,40.862901,32.178981,27.939465,29.324584,23.885710,24.887880,16.432391,45.570680,38.173687,...,4.347437,4.101772,2.987192,3.862508,3.362597,2.566516,3.049593,5.025746,1.893471,8.942586
2,31.731240,86.587706,60.131360,57.718046,58.326412,54.090394,50.138146,35.408905,94.937737,90.762179,...,6.332389,5.479924,5.946744,6.017439,5.585987,3.870925,4.083335,7.451925,2.946266,3.868906


In [77]:
df_n = quantileNormalize(test)

In [81]:
df_n.head(10)

,1,2,3,4,5,6,7,8,9,10,...,515,516,517,518,519,520,521,522,523,524
0,,,,,,,,,,,,,,,,,,,,,
1,8.788037,8.788037,8.788037,8.788037,8.788037,8.788037,8.788037,8.788037,8.788037,8.788037,...,8.788037,11.735517,8.788037,8.788037,8.788037,8.788037,8.788037,11.735517,8.788037,17.842278
2,13.123562,11.735517,17.842278,17.842278,11.735517,17.842278,13.123562,14.893546,17.842278,17.842278,...,13.123562,17.842278,17.842278,13.123562,11.735517,17.842278,13.123562,14.893546,14.893546,11.735517
3,14.893546,17.842278,13.123562,11.735517,17.842278,14.893546,14.893546,13.123562,14.893546,14.893546,...,14.893546,14.893546,13.123562,14.893546,17.842278,14.893546,11.735517,17.842278,13.123562,14.893546
4,11.735517,13.123562,14.893546,13.123562,13.123562,11.735517,11.735517,11.735517,13.123562,13.123562,...,17.842278,13.123562,11.735517,17.842278,13.123562,13.123562,14.893546,13.123562,11.735517,8.788037
5,17.842278,14.893546,11.735517,14.893546,14.893546,13.123562,17.842278,17.842278,11.735517,11.735517,...,11.735517,8.788037,14.893546,11.735517,14.893546,11.735517,17.842278,8.788037,17.842278,13.123562


In [84]:
df_n = df_n.round(6)

In [85]:
df_n.to_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/qn_exons_matrix.csv", sep=",", header=None)

In [110]:
GeneExp = pd.read_csv("../data/expression/brainspan/gene_matrix/expression_matrix.csv", header=None, index_col=0)
display(GeneExp.head(2))
df_n = quantileNormalize(GeneExp)
df_n = df_n.round(6)
df_n.to_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/qn.expression_matrix.csv", sep=",", header=None)

,1,2,3,4,5,6,7,8,9,10,...,515,516,517,518,519,520,521,522,523,524
0,,,,,,,,,,,,,,,,,,,,,
1,36.447128,24.251253,19.330479,27.668607,19.998231,14.680673,27.548101,16.580183,44.587799,44.943915,...,2.320932,1.781548,2.277359,1.832737,1.555696,2.081944,3.484685,4.816781,3.034464,3.082820
2,0.044081,0.067338,0.000000,0.145466,0.185188,0.311180,0.000000,0.000000,0.473831,0.181220,...,0.758571,0.000000,0.061869,0.026876,0.100691,0.140675,0.300576,0.126526,0.000000,0.424134


In [78]:
ins = BrainSpan()
bp_exon_col_meta = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/columns_metadata.csv", delimiter=",")
bp_exon_col_meta.age.unique()
bp_exon_col_meta["Period"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[0], axis=1)
bp_exon_col_meta["Stage"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[1], axis=1)
bp_exon_col_meta["Description"] = bp_exon_col_meta.apply(lambda row: ins.TemporalMap(row["age"])[2], axis=1)
# Rows Meta Data
bp_exon_row_meta = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/rows_metadata.csv", delimiter=",")
bp_exon_row_meta["exon length"] = bp_exon_row_meta["end"] - bp_exon_row_meta["start"]
display(bp_exon_row_meta.head(2))
ExonExp = pd.read_csv("../data/expression/brainspan/exons_matrix/qn_exons_matrix.csv", header=None)
display(ExonExp.head(2))

,row_num,gene_id,ensembl_gene_id,gene_symbol,entrez_id,start,end,exon length
0,1,1086864,ENSG00000000003,TSPAN6,7105.0,99883666,99884983,1317
1,2,1086864,ENSG00000000003,TSPAN6,7105.0,99885755,99885863,108


,0,1,2,3,4,5,6,7,8,9,...,515,516,517,518,519,520,521,522,523,524
0,1,12.147751,39.243277,28.518602,23.230875,26.230301,20.025094,19.736859,16.195879,40.864157,...,6.641969,6.717663,2.946814,5.231312,3.525105,2.775830,4.319969,9.799409,1.883989,21.652156
1,2,29.148381,82.511717,55.319827,51.980301,53.281452,49.598084,44.207035,33.799339,88.398147,...,9.013514,8.462362,5.768021,7.981446,6.169699,4.133613,5.709670,13.147673,2.943256,12.461572


In [296]:
Regionsgt20 = ['OFC', 'VFC', 'HIP', 'ITC', 'AMY', 'DFC', 'STC', 'MFC', 'STR', 'IPC', 
            'V1C', 'S1C', 'A1C', 'M1C', 'CBC', 'MD']

In [297]:
GeneExp = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/qn.expression_matrix.csv", header=None)
GeneRow = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/rows_metadata.csv")
GeneCol = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/columns_metadata.csv")
GeneCol["Period"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[0], axis=1)
GeneCol["Stage"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[1], axis=1)
GeneCol["Description"] = GeneCol.apply(lambda row: ins.TemporalMap(row["age"])[2], axis=1)
GeneDat = [GeneExp, GeneRow, GeneCol]
gene_stage2column_num = {}
for stage in ins.Stages:
    column_num = set(GeneCol[GeneCol["Period"]==stage]["column_num"].values)
    gene_stage2column_num[stage] = column_num

In [299]:
bp_exon_row_meta_lgdpre_with_gene = pd.read_excel("./data/ProLGD_wilger2014_predicted.July20.xlsx")
#bp_exon_row_meta_lgdpre_with_gene = bp_exon_row_meta_lgdpre_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_lgdpre_with_gene["effectGene"].values))
print(len(Genes))
ins.LoadGeneSetData2Fil2("qn.ProbandLGD_PRE.Regionsgt20.98Genes.GeneLog2Exp.csv", 
                        Genes, Regionsgt20, GeneRow, GeneCol, GeneExp)

98
Total Num of Genes: 98


/Users/jiayao/Work/BrainDisorders/src/utils.py:1467: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  exp = Matrix.get_value(GeneID-1, col)


In [175]:
94 * 12

1128

In [80]:
bp_exon_row_meta_syn_with_gene = pd.read_excel("GenesExonsTargetedBySYN.xlsx", index=False)
bp_exon_row_meta_syn_with_gene = bp_exon_row_meta_syn_with_gene.replace(np.nan, '', regex=True)

IOError: [Errno 2] No such file or directory: 'GenesExonsTargetedBySYN.xlsx'

In [99]:
bp_exon_row_meta_syn_with_gene.head()

,row_num,gene_id,ensembl_gene_id,gene_symbol,entrez_id,start,end,exon length,cds length,NVIQ70,VIQ70,Gender,Func,Vars,GeneHited,Last
0,137,1086873,ENSG00000001167,NFYA,4800,41040683,41040823,140,140,,,,,,T,F
1,138,1086873,ENSG00000001167,NFYA,4800,41046767,41046903,136,75,,,,,,T,F
2,139,1086873,ENSG00000001167,NFYA,4800,41048549,41048636,87,87,,,,,,T,F
3,140,1086873,ENSG00000001167,NFYA,4800,41051784,41051931,147,147,,,,,,T,F
4,141,1086873,ENSG00000001167,NFYA,4800,41057317,41057449,132,132,,,,,,T,F


In [66]:
ins = BrainSpan()

In [100]:
Genes = list(set(bp_exon_row_meta_syn_with_gene["gene_symbol"].values))
ins.LoadGeneSetData2Fil("qn.ProbandSYN.Regionsgt20.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta_syn_with_gene, bp_exon_col_meta, ExonExp)

Total Num of Genes: 593


In [101]:
bp_exon_row_meta_mis_with_gene = pd.read_excel("GenesExonsTargetedByMIS.xlsx", index=False)
bp_exon_row_meta_mis_with_gene = bp_exon_row_meta_mis_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_mis_with_gene["gene_symbol"].values))
ins.LoadGeneSetData2Fil("qn.ProbandMIS.Regionsgt20.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta_mis_with_gene, bp_exon_col_meta, ExonExp)

Total Num of Genes: 1436


In [102]:
bp_exon_row_meta_sibsyn_with_gene = pd.read_excel("GenesExonsTargetedBySibSYN.xlsx", index=False)
bp_exon_row_meta_sibsyn_with_gene = bp_exon_row_meta_sibsyn_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_sibsyn_with_gene["gene_symbol"].values))
ins.LoadGeneSetData2Fil("qn.SiblingSYN.Regionsgt20.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta_sibsyn_with_gene, bp_exon_col_meta, ExonExp)

Total Num of Genes: 452


In [104]:
bp_exon_row_meta_siblgd_with_gene = pd.read_excel("GenesExonsTargetedBySibLGD.xlsx", index=False)
bp_exon_row_meta_siblgd_with_gene = bp_exon_row_meta_siblgd_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_siblgd_with_gene["gene_symbol"].values))
ins.LoadGeneSetData2Fil("qn.SinglingLGD.Regionsgt20.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta_siblgd_with_gene, bp_exon_col_meta, ExonExp)

Total Num of Genes: 176


In [309]:
#bp_exon_row_meta_lgdpre_with_gene = pd.read_excel("GenesExonsTargetedByLGD_SCZ_predicted.xlsx", index=False)
#bp_exon_row_meta_lgdpre_with_gene = bp_exon_row_meta_lgdpre_with_gene.replace(np.nan, '', regex=True)
#Genes = list(set(bp_exon_row_meta_lgdpre_with_gene["gene_symbol"].values))
bp_exon_row_meta_lgdpre_with_gene = pd.read_excel("./data/ProLGD_wilger2014_predicted.July20.xlsx")
#bp_exon_row_meta_lgdpre_with_gene = bp_exon_row_meta_lgdpre_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_lgdpre_with_gene["effectGene"].values))
print(len(Genes))
ins.LoadGeneSetData2Fil("qn.ProbandLGD_PRE.Regionsgt20.98Genes.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta, bp_exon_col_meta, qnExonExp)

98
Total Num of Genes: 98


/Users/jiayao/Work/BrainDisorders/src/utils.py:1446: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  exp = Matrix.get_value(ExonID-1, col)


In [106]:
bp_exon_row_meta_scz_with_gene = pd.read_excel("GenesExonsTargetedByLGD_SCZ.xlsx", index=False)
bp_exon_row_meta_scz_with_gene = bp_exon_row_meta_scz_with_gene.replace(np.nan, '', regex=True)
Genes = list(set(bp_exon_row_meta_scz_with_gene["gene_symbol"].values))
ins.LoadGeneSetData2Fil("qn.SCZ.Regionsgt20.GeneExonLog2Exp.csv", 
                        Genes, Regionsgt20, bp_exon_row_meta_scz_with_gene, bp_exon_col_meta, ExonExp)

Total Num of Genes: 553


# Get bed file of genomics regions associated with Austism

In [118]:
bp_exon_row_meta_lgd_with_gene = pd.read_excel("./data/GenesExonsTargetedByLGD_wilger2014_predicted.xlsx", index=False)
bp_exon_row_meta_lgd_with_gene = bp_exon_row_meta_lgd_with_gene.replace(np.nan, '', regex=True)
bp_exon_row_meta_lgd_with_gene.head(20)
Genes = set(bp_exon_row_meta_lgd_with_gene["gene_symbol"])
print Genes, len(Genes)

set([u'HECTD1', u'PCSK2', u'CASP8AP2', u'TNRC6B', u'L1CAM', u'GOLGA5', u'DYRK1A', u'CUL3', u'JAKMIP1', u'SLC25A6', u'IRF2BPL', u'ATP1B1', u'NMT1', u'UBR5', u'ARID1B', u'NRXN1', u'TERF2', u'WNT7B', u'FOXP1', u'ADNP', u'ELAVL2', u'PHF21A', u'KAT2B', u'ARHGAP5', u'NCKAP1', u'SPEN', u'CHD2', u'UBAP2L', u'CHD8', u'TNRC18', u'NBEA', u'CDC42BPB', u'KATNAL2', u'PAX5', u'SYNGAP1', u'MED13L', u'WDFY3', u'MBD5', u'PHF2', u'PHF3', u'DSCAML1', u'SRCAP', u'GIGYF2', u'ZFHX3', u'CSMD2', u'CBX4', u'NEDD9', u'THSD7A', u'MSL2', u'KDM6B', u'DOT1L', u'PDCD1', u'ASH1L', u'LARP4B', u'WAC', u'TBR1', u'DLL1', u'BTAF1', u'XKR6', u'EIF2AK2', u'ATP1A1', u'CIC', u'DIP2A', u'BCL11A', u'CTNNB1', u'ZMYND11', u'DSCAM', u'UBN2', u'SPTBN1', u'ANKRD11', u'POGZ', u'LTN1', u'AHDC1', u'RAI1', u'SCN2A', u'MED13', u'CNOT3', u'NACC1', u'INTS6', u'TANC2', u'VCP', u'SUFU', u'TRIP12', u'SHANK2', u'NFIB', u'NFIA', u'SPAST', u'NOTCH1', u'C11orf24', u'GRIN2B', u'ANK2', u'MAP3K14', u'KAT6A', u'ILF2']) 94


In [118]:
GenCodeV10Fil = "/Users/jiayao/Work/Resources/gencode.v10.annotation.gtf"
Fout = "ASD.Genes.raw.bed"
writer = csv.writer(open(Fout, 'wb'), delimiter="\t")
for l in open(GenCodeV10Fil):
    if l.startswith("#"):
        continue
    llist = l.strip().split("\t")  
    info = llist[8].split(";")
    gene = info[4].split()[-1].strip('"')
    if gene in Genes:
        Chr, start, end = llist[0].strip("Chr"), llist[3], llist[4]
        writer.writerow([Chr, start, end, gene])

# Extract neurotypical individuals LGD variants from gnomAD

In [4]:
CSQ_FMT = 'Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|ALLELE_NUM|DISTANCE|STRAND|FLAGS|VARIANT_CLASS|MINIMISED|SYMBOL_SOURCE|HGNC_ID|CANONICAL|TSL|APPRIS|CCDS|ENSP|SWISSPROT|TREMBL|UNIPARC|GENE_PHENO|SIFT|PolyPhen|DOMAINS|HGVS_OFFSET|GMAF|AFR_MAF|AMR_MAF|EAS_MAF|EUR_MAF|SAS_MAF|AA_MAF|EA_MAF|ExAC_MAF|ExAC_Adj_MAF|ExAC_AFR_MAF|ExAC_AMR_MAF|ExAC_EAS_MAF|ExAC_FIN_MAF|ExAC_NFE_MAF|ExAC_OTH_MAF|ExAC_SAS_MAF|CLIN_SIG|SOMATIC|PHENO|PUBMED|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|LoF|LoF_filter|LoF_flags|LoF_info'
CSQ_FMT = CSQ_FMT.split("|")
print CSQ_FMT
def getINFO(info_string):
    infolist = info_string.split(';')
    infodict = {}
    for kv in infolist:
        kv = kv.split('=')
        if len(kv) == 2:
            k, v = kv
            if k in infodict:
                infodict[k].append(v) 
            else:
                infodict[k] = [v]
    return infodict
def searchCANONICAL(CSQLIST):
    for csq in CSQLIST:
        csq = csq.split("|")
        CSQ = dict(zip(CSQ_FMT, csq))
        if CSQ["CANONICAL"] == "YES":
            #return csq
            return "YES"
    return None
def displayCSQ(head, _dict):
    for item in head:
        print "{}:{}\t".format(item, _dict[item]),

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'ALLELE_NUM', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'MINIMISED', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'DOMAINS', 'HGVS_OFFSET', 'GMAF', 'AFR_MAF', 'AMR_MAF', 'EAS_MAF', 'EUR_MAF', 'SAS_MAF', 'AA_MAF', 'EA_MAF', 'ExAC_MAF', 'ExAC_Adj_MAF', 'ExAC_AFR_MAF', 'ExAC_AMR_MAF', 'ExAC_EAS_MAF', 'ExAC_FIN_MAF', 'ExAC_NFE_MAF', 'ExAC_OTH_MAF', 'ExAC_SAS_MAF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'LoF', 'LoF_filter', 'LoF_flags', 'LoF_info']


In [5]:
LGD_class = set(["stoploss", "stopgain", "frameshift_insertion", "frameshift_deletion"])
print LGD_class

set(['stopgain', 'frameshift_insertion', 'stoploss', 'frameshift_deletion'])


In [23]:
import gzip
gnomad_file = "../data/gnomad.exomes.ASD.genes.vcf.gz"
hand = gzip.open(gnomad_file, 'rb')
outfil = csv.writer(open("benigh.tsv", 'wb'), delimiter="\t")
fout2 = open("benign.vcf", 'wb')
#outfil.writerow(head)
for l in hand:
    if l.startswith("#"):
        fout2.write(l)
        continue
    llist = l.strip().split("\t")
    if llist[6] != "PASS":
        continue
    Chr, Pos, ID, ref, alts = llist[0:5]
    INFO_string = llist[7]
    info_dict = getINFO(INFO_string)
    for i,alt in enumerate(alts.split(',')):
        #if info_dict["AC_Female"] == 0:
        #    continue
        exoincfunc = info_dict["ExonicFunc.refGene"][i]
        AF = info_dict["gnomAD_exome_ALL"][i]
        Gene = info_dict["Gene.refGene"][i]
        AC_male = info_dict["AC_Male"][0].split(",")[i]
        AC_female = info_dict["AC_Female"][0].split(",")[i]
        if exoincfunc in LGD_class:
            outfil.writerow([Chr, Pos, ref, alt, Gene, exoincfunc, AF, AC_male, AC_female])
            fout2.write(l)
        else:
            continue

In [146]:
print LGD_class #AC_Male AC_Female

set(['stopgain', 'frameshift_insertion', 'stoploss', 'frameshift_deletion'])


In [24]:
outfil = pd.read_csv("benigh.tsv",  delimiter="\t")

In [29]:
T1 = outfil[outfil["AF"]!=0]
print T1.shape

(1160, 9)


In [30]:
T2 = T1[T1["AC_Male"]!=0]
T3 = T1[T1["AC_Female"]!=0]
print T2.shape, T3.shape

(684, 9) (580, 9)


In [34]:
T1.to_csv("../data/benign.all.tsv", index=False, sep="\t")
T2.to_csv("../data/benign.male.tsv", index=False, sep="\t")
T3.to_csv("../data/benign.female.tsv", index=False, sep="\t")

In [56]:
import gzip
gnomad_file = "../data/ExAC.r0.3.nonpsych.ASD.genes.vcf.hg19_multianno.vcf.gz"
hand = gzip.open(gnomad_file, 'rb')
outfil = csv.writer(open("../data/benign.AC1.nonssych.tsv", 'wb'), delimiter="\t")
outfil.writerow(["#Chr","Pos","Ref","Alt","Gene","Consequence","AF", "AC"])
for l in hand:
    if l.startswith("#"):
        fout2.write(l)
        continue
    llist = l.strip().split("\t")
    if llist[6] != "PASS":
        continue
    Chr, Pos, ID, ref, alts = llist[0:5]
    INFO_string = llist[7]
    info_dict = getINFO(INFO_string)
    for i,alt in enumerate(alts.split(',')):
        #if info_dict["AC_Female"] == 0:
        #    continue
        exoincfunc = info_dict["ExonicFunc.refGene"][i]
        AF = info_dict["gnomAD_exome_ALL"][i]
        Gene = info_dict["Gene.refGene"][i]
        AC = info_dict["AC"][0].split(",")[i]
        if int(AC) != 1:
            continue
        #AC_male = info_dict["AC_Male"][0].split(",")[i]
        #AC_female = info_dict["AC_Female"][0].split(",")[i]
        if exoincfunc in LGD_class:
            outfil.writerow([Chr, Pos, ref, alt, Gene, exoincfunc, AF, AC])
        else:
            continue

In [120]:
print Genes, len(Genes)

set([u'HECTD1', u'PCSK2', u'CASP8AP2', u'TNRC6B', u'L1CAM', u'GOLGA5', u'DYRK1A', u'CUL3', u'JAKMIP1', u'SLC25A6', u'IRF2BPL', u'ATP1B1', u'NMT1', u'UBR5', u'ARID1B', u'NRXN1', u'TERF2', u'WNT7B', u'FOXP1', u'ADNP', u'ELAVL2', u'PHF21A', u'KAT2B', u'ARHGAP5', u'NCKAP1', u'SPEN', u'CHD2', u'UBAP2L', u'CHD8', u'TNRC18', u'NBEA', u'CDC42BPB', u'KATNAL2', u'PAX5', u'SYNGAP1', u'MED13L', u'WDFY3', u'MBD5', u'PHF2', u'PHF3', u'DSCAML1', u'SRCAP', u'GIGYF2', u'ZFHX3', u'CSMD2', u'CBX4', u'NEDD9', u'THSD7A', u'MSL2', u'KDM6B', u'DOT1L', u'PDCD1', u'ASH1L', u'LARP4B', u'WAC', u'TBR1', u'DLL1', u'BTAF1', u'XKR6', u'EIF2AK2', u'ATP1A1', u'CIC', u'DIP2A', u'BCL11A', u'CTNNB1', u'ZMYND11', u'DSCAM', u'UBN2', u'SPTBN1', u'ANKRD11', u'POGZ', u'LTN1', u'AHDC1', u'RAI1', u'SCN2A', u'MED13', u'CNOT3', u'NACC1', u'INTS6', u'TANC2', u'VCP', u'SUFU', u'TRIP12', u'SHANK2', u'NFIB', u'NFIA', u'SPAST', u'NOTCH1', u'C11orf24', u'GRIN2B', u'ANK2', u'MAP3K14', u'KAT6A', u'ILF2']) 94


In [123]:
def parseMC(MC):
    return MC.split("|")[1].split(",")[0].split(",")[0]
def parseGENEINFO(GENEINFO):
    return GENEINFO.split(":")[0]
#predicted_genes = set([x.strip() for x in file()])
gnomad_file = "../data/clinvar_20190108.vcf"
hand = open(gnomad_file, 'rb')
outfil = csv.writer(open("../data/clinvar.predicted.genes.tsv", 'wb'), delimiter="\t")
outfil.writerow(["#Chr","Pos","Ref","Alt","Gene","Consequence","AF", "CLNSIG", "CLNDN"])
for l in hand:
    if l.startswith("#"):
        fout2.write(l)
        continue
    llist = l.strip().split("\t")

    Chr, Pos, ID, ref, alts = llist[0:5]
    INFO_string = llist[7]
    info_dict = getINFO(INFO_string)
    for i,alt in enumerate(alts.split(',')):
        #if info_dict["AC_Female"] == 0:
        #    continue
        try:
            exoincfunc = parseMC(info_dict["MC"][i])
        except:
            exoincfunc = "."
        try:
            AF = info_dict["AF_EXAC"][i]
        except:
            AF = '0'
        try:
            Gene = parseGENEINFO(info_dict["GENEINFO"][i])
        except:
            Gene = "."
        if Gene not in Genes:
            continue
        try:
            CLNSIG = info_dict["CLNSIG"][i]
        except:
            CLNSIG = '.'
        try:
            CLNDN = info_dict["CLNDN"][i]
        except:
            CLNDN = "."

        #if exoincfunc in LGD_class:
        #    outfil.writerow([Chr, Pos, ref, alt, Gene, exoincfunc, AF, AC])
        #else:
        #    continue
        outfil.writerow([Chr, Pos, ref, alt, Gene, exoincfunc, AF, CLNSIG, CLNDN])

In [134]:
df = pd.read_csv("../data/clinvar.predicted.genes.tsv", sep="\t")
clinvar_lgd = set(["frameshift_variant", "nonsense", "splice_acceptor_variant", "splice_donor_variant"])
df1 = df[(df["Consequence"].isin(clinvar_lgd)) & (df["CLNSIG"].isin(["Pathogenic", "Likely_pathogenic"]))]
df1 = df1[df["CLNDN"]!="not_provided"]
print df1.shape
df1.to_csv("../data/DenovoVariants/clinvar.predicted.path.lgd.tsv", index=False, sep="\t")

(672, 9)


/Users/jiayao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [133]:
df = pd.read_csv("../data/clinvar.predicted.genes.tsv", sep="\t")
clinvar_lgd = set(["frameshift_variant", "nonsense", "splice_acceptor_variant", "splice_donor_variant"])
df1 = df[(df["Consequence"].isin(clinvar_lgd)) & (df["CLNSIG"].isin(["Benign", "Likely_benign", "Benign/Likely_benign"]))]
#df1 = df1[df["CLNDN"]=="not_provided"]
print df1.shape
df1.to_csv("../data/DenovoVariants/clinvar.predicted.benign.lgd.tsv", index=False, sep="\t")

(8, 9)


In [135]:
print set([x.split("|")[0] for x in df1["CLNDN"].values])

set(['Seizures', 'IRF2BPL-related_condition', 'Hydrocephalus_due_to_aqueductal_stenosis', 'Neurodevelopmental_disorder', 'Hydrocephalus,_X-linked,_with_hirschsprung_disease', 'KBG_syndrome', 'White-sutton_syndrome', 'Wide_mouth', 'Inborn_genetic_diseases', 'Autism_17', 'Early_infantile_epileptic_encephalopathy_11', 'Mental_retardation,_autosomal_dominant_5', 'Helsmoortel-van_der_aa_syndrome', 'NEURODEVELOPMENTAL_DISORDER_WITH_REGRESSION,_ABNORMAL_MOVEMENTS,_LOSS_OF_SPEECH,_AND_SEIZURES', 'Adams-Oliver_syndrome_5', 'Medulloblastoma_with_extensive_nodularity', 'Autism_spectrum_disorder', 'Desanto-shinawi_syndrome', 'Heart,_malformation_of', 'Macrocephalus', 'Pseudohypoaldosteronism,_type_2', 'Benign_familial_neonatal-infantile_seizures', 'Abdominal_obesity-metabolic_syndrome_3', 'Delayed_speech_and_language_development', 'Mental_retardation,_autosomal_dominant_7', 'intellectual_deficiency', 'Hearing_impairment', 'MENTAL_RETARDATION,_AUTOSOMAL_DOMINANT_52', 'Vesicoureteral_reflux', 'Thick

In [100]:
df = pd.read_excel("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/denovodb.schizophrenia.xlsx")
df = df.drop_duplicates(["#SampleID","Chr","Position"])
df.to_csv("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/denovodb.schizophrenia.dedup.tsv", sep="\t")

In [101]:
df.shape

(596, 31)

In [161]:
maps = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/rows_metadata.csv")
maps = dict(zip(maps["entrez_id"].values, maps["gene_symbol"].values))
scz_genes = [x.strip() for x in file("/Users/jiayao/Work/BrainDisorders/data/scz-predicted-wiglerlike.txt")]
print scz_genes
scz_genes = [maps[int(x)] for x in scz_genes if int(x) in maps]
print scz_genes

['83593', '9874', '91746', '9140', '5660', '4288', '1833', '55269', '3320', '259173', '3008', '7862', '3655', '83715', '9829', '27252', '83879', '316', '1951', '205717', '84343', '27284', '51663', '7915', '8294', '22999', '56479', '5649', '23303', '6713', '26147', '54932', '51703', '5140', '9415', '116985', '726', '55572', '3888', '56890', '79023', '57523', '87', '4308', '4350', '1327', '8943', '10994', '23152', '7052', '5116', '2033', '84061', '3667', '79142', '6002', '161', '84647', '124944', '8857', '28981', '1662', '821', '55748', '11200', '26586']
['RASSF5', 'TLK1', 'YTHDC1', 'ATG12', 'PSAP', 'MKI67', 'EPYC', 'PSPC1', 'HSP90AA1', 'ALS2CL', 'HIST1H1E', 'BRPF1', 'ITGA6', 'ESPN', 'DNAJC6', 'KLHL20', 'CDCA7', 'AOX1', 'CELSR3', 'KIAA2018', 'HPS3', 'SULT1B1', 'ZFR', 'ALDH5A1', 'HIST1H4I', 'RIMS1', 'KCNQ5', 'RELN', 'KIF13B', 'SQLE', 'PHF19', 'EXD3', 'ACSL5', 'PDE3B', 'FADS2', 'ARAP1', 'CAPN5', 'FOXRED1', 'KRT82', 'MDM1', 'NUP37', 'NYNRIN', 'ACTN1', 'TRPM1', 'MPG', 'COX4I1', 'AP3D1', 'ILV

In [162]:
with open("/Users/jiayao/Work/BrainDisorders/data/scz-predicted-wiglerlike.genesymbol.txt", 'wb') as f:
    for gene in scz_genes:
        f.write(gene+"\n")

In [151]:
df = pd.read_csv("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/denovo-db.non-ssc-samples.variants.tsv", sep="\t")
df = df[df["Gene"].isin(Genes)]

In [150]:
LGD = [   "frameshift", "frameshift-near-splice", 
          "splice-acceptor", "splice-donor",
          "start-lost", "stop-gained", "stop-gained-near-splice", "stop-lost"]
df = df[df["PrimaryPhenotype"]!="autism"]
#df = df[df["PrimaryPhenotype"]!="intellectualDisability"]
df = df[df["FunctionClass"].isin(LGD)]
df = df.drop_duplicates(["#SampleID","Chr","Position"])
print df.shape
df.to_csv("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/denovo-db.non-autism.predicted.genes.lgd.tsv", index="False", sep="\t")

(241, 31)


In [152]:
LGD = [   "frameshift", "frameshift-near-splice", 
          "splice-acceptor", "splice-donor",
          "start-lost", "stop-gained", "stop-gained-near-splice", "stop-lost"]
#df = df[df["PrimaryPhenotype"]!="autism"]
df = df[df["PrimaryPhenotype"]=="intellectualDisability"]
df = df[df["FunctionClass"].isin(LGD)]
df = df.drop_duplicates(["#SampleID","Chr","Position"])
print df.shape
df.to_csv("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/denovo-db.ID.predicted.genes.lgd.tsv", index="False", sep="\t")

(56, 31)


In [7]:
Variants = pd.read_excel("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/wigler2014ProLGD.xlsx")
Variants.loc[Variants["effectGene"]=="KMT2E", "effectGene"] = "MLL5" 
LGDGenes = set(Variants["effectGene"].values)
print(len(LGDGenes))

352


In [10]:
"MLL5" in LGDGenes

True

In [8]:
qn_gene_exp = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/qn.expression_matrix.csv", 
    header=None, names=[x if x>0 else "index" for x in range(525)])
gene_exp_col = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/columns_metadata.csv")
gene_exp_row = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/gene_matrix/rows_metadata.csv")
gene_exp_row_recgenes = gene_exp_row[gene_exp_row["gene_symbol"].isin(LGDGenes)]
qn_gene_exp_recgenes = qn_gene_exp[qn_gene_exp["index"].isin(gene_exp_row_recgenes["row_num"].values)]
qn_gene_exp_recgenes.to_csv("unifiedmodel/LGDGenes_exp_matrix.csv", index=False)

In [9]:
qn_exon_exp = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/qn_exons_matrix.csv", 
    header=None, names=[x if x>0 else "index" for x in range(525)])
exon_exp_col = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/columns_metadata.csv")
exon_exp_row = pd.read_csv(
    "/Users/jiayao/Work/BrainDisorders/data/expression/brainspan/exons_matrix/rows_metadata.csv")
exon_exp_row_recgenes = exon_exp_row[exon_exp_row["gene_symbol"].isin(LGDGenes)]
qn_exon_exp_recgenes = qn_exon_exp[qn_exon_exp["index"].isin(exon_exp_row_recgenes["row_num"].values)]
qn_exon_exp_recgenes.to_csv("unifiedmodel/LGDExons_exp_matrix.csv", index=False)

In [ ]:
#Get IQ of each genes
Rs = []
for i in range(5000):
    ALL_RelExp = np.array([])
    ALL_Normed_IQD = np.array([])
    for gene in Genes:
        df = Variants[Variants["effectGene"]==gene]
        exon_df = exon_exp_row_recgenes[exon_exp_row_recgenes["gene_symbol"]==gene]
        population, weights = GetExonProb(exon_df)
        IQs = [max(0, (100-x)) for x in df["NVIQ"].values]
        rel_exps = []
        for i in range(len(IQs)):
            idx = choice(population, weights)
            rel_exps.append(exon_exp_row_recgenes.at[idx, "rel.exp"])
        regr = regGene(rel_exps, IQs)
        slope = regr.coef_[0]
        ALL_RelExp = np.append(ALL_RelExp, rel_exps)
        NormIQDs = np.array([x/slope for x in IQs])
        ALL_Normed_IQD = np.append(ALL_Normed_IQD, NormIQDs)
    model = linear_model.LinearRegression(fit_intercept=False)
    ALL_RelExp = ALL_RelExp.reshape(-1,1)
    R, P = scipy.stats.pearsonr([x[0] for x in ALL_RelExp], ALL_Normed_IQD) 
    Rs.append(R)

In [ ]:
Rs = sorted(Rs)
R = 0.629
pvalue = 0
for i,v in enumerate(Rs):
    if v > R:
        break
pvalue = 1 - i/5000
print(pvalue)

In [19]:
class GTFRecord:
    def __init__(self, Chr, source, Type, start, end, strand, info):
        self.Chr = Chr
        self.source = source
        self.Type = Type
        self.start = start
        self.end = end
        self.strand = strand
        self.info = info

class GTFGene:
    def __init__(self, GeneName, GeneID, strand):
        self.GeneName = GeneName
        self.GeneID = GeneID
        self.strand = strand
        self.Transcripts = {}

class GTFTranscript:
    def __init__(self, gene, TranscriptID, TranscriptName, strand):
        self.TranscriptID = TranscriptID
        self.TranscriptName = TranscriptName
        self.GeneName = gene
        self.Exons = []
        self.strand = strand
    def LastExonJunction(self): # Interval of last exon and 2nd-last 55nt EEJ, tuple of tuple
        if self.strand == "+":
            LEJ = (self.Exons[-2].end - 55, self.Exons[-2].end)
            LE = (self.Exons[-1].start, self.Exons[-1].end)
        elif self.strand == "-":
            LEJ = (self.Exons[-2].start, self.Exons[-2].start + 50)
            LE = (self.Exons[-1].start, self.Exons[-1].end)
        return (LE, LEJ)
        
class GTFExon:
    def __init__(self, start, end, TranscriptID, strand):
        self.TranscriptID = TranscriptID
        self.start = int(start)
        self.end = int(end)
        self.strand = strand
        
def gtf_info_parser(info):
    res = {}
    for term in info.split(";"):
        if term == "":
            continue
        # print(">",term)
        key, v = term.split()
        v = v.strip('"')
        res[key] = v
    return res
            
def LoadGeneCode(genecodefil):
    Genes = {} 
    Transcripts = {}
    hand = open(genecodefil, 'rt')
    for l in hand:
        if l.startswith("#"):
            continue
        llist = l.strip().split("\t")
        info = gtf_info_parser(llist[8]) 
        if llist[2] == "gene":
            Genes[info["gene_name"]] = GTFRecord(
                llist[0], llist[1], llist[2], llist[3], llist[4], llist[6], info)
            Transcripts[info["gene_name"]] = []
        elif llist[2] == "transcript":
            if info["gene_name"] not in Genes:
                Genes[info["gene_name"]] = GTFRecord(
                    llist[0], llist[1], llist[2], llist[3], llist[4], llist[6], info)
                Transcripts[info["gene_name"]] = []
            Transcripts[info["gene_name"]].append(
                GTFRecord(llist[0], llist[1], llist[2], llist[3], llist[4], llist[6], info))
    return Genes, Transcripts

In [56]:
Variants = pd.read_excel("/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/wigler2014ProLGD.xlsx")
Variants.loc[Variants["effectGene"]=="KMT2E", "effectGene"] = "MLL5" 
GeneCount = Variants.groupby("effectGene")["effectGene"].count()
Variants["GeneCount"] = Variants.apply(lambda row: GeneCount[row["effectGene"]], axis=1)
Variants = Variants[Variants["GeneCount"]>=2]
LGDGenes = set(Variants["effectGene"].values)
print(len(LGDGenes))
LGDGenes.add("KMT2E")

28


In [57]:
gtf = "/Users/jiayao/Work/BrainDisorders/data/gencode.v19.annotation.gtf"
out = open("unifiedmodel/RecLGDgenes.gencode.v19.gtf", 'wt')
hand = open(gtf, 'rt')
for l in hand:
    if l.startswith("#"):
        out.write(l)
        continue
    llist = l.strip().split("\t")
    info = gtf_info_parser(llist[8]) 
    if info["gene_name"] in LGDGenes:
        out.write(l)

In [5]:
df = pd.read_csv("unifiedmodel/appris_data.principal.txt", delimiter="\t", header=None)
df = df[df[4].isin(["PRINCIPAL:1","PRINCIPAL:2","PRINCIPAL:3"])]
principal_trans = set(df[2].values)

In [44]:

Genes = {}
hand = open("unifiedmodel/RecLGDgenes.gencode.v10.gtf", 'rt')
for l in hand:
    if l.startswith("#"):
        continue
    llist = l.strip().split("\t")
    info = gtf_info_parser(llist[8]) 
    CHR = llist[0].lstrip("chr")
    strand = llist[6]
    start = int(llist[3])
    end = int(llist[4])
    if llist[2] == "gene":
        gene_name = info["gene_name"]
        gene_id = info["gene_id"]
        Genes[gene_name] = GTFGene(gene_name, gene_id, strand)
    elif llist[2] == "transcript":
        gene_name = info["gene_name"]
        gene_id = info["gene_id"]
        transcript_name = info["transcript_id"]
        transcript_id = info["transcript_id"]
        transcript_id = transcript_id.split(".")[0]
        transcript_type = info["transcript_type"]
        #if transcript_id not in Genes[gene_name].Transcripts and transcript_id in principal_trans:
        if transcript_id not in Genes[gene_name].Transcripts and transcript_type=="protein_coding":
            Genes[gene_name].Transcripts[transcript_id] = GTFTranscript(gene_name, transcript_name, transcript_id, strand)
    elif llist[2] == "exon":
        gene_name = info["gene_name"]
        gene_id = info["gene_id"]
        transcript_name = info["transcript_id"]
        transcript_id = transcript_id.split(".")[0]
        #if transcript_id in principal_trans:
        #transcript_type = info["transcript_type"]
        if transcript_type=="protein_coding":
            #exon_id = info["exon_id"]
            #exon= GTFExon(exon_id, start, end, transcript_id, strand)
            exon= GTFExon("1", start, end, transcript_id, strand)
            Genes[gene_name].Transcripts[transcript_id].Exons.append(exon)

In [39]:
gene_obj = Genes["CHD8"]
for k, v in gene_obj.Transcripts.items():
    if len(v.Exons) >= 2:
        print(k, v.strand, v.LastExonJunction())

ENST00000430710 - ((21853353, 21854335), (21859106, 21859161))
ENST00000399982 - ((21853354, 21854335), (21859106, 21859161))
ENST00000557364 - ((21853645, 21854335), (21859106, 21859161))
ENST00000553870 - ((21854170, 21854335), (21859106, 21859161))
ENST00000555935 - ((21867765, 21867864), (21868140, 21868195))
ENST00000553283 - ((21897296, 21897494), (21905060, 21905115))
ENST00000553622 - ((21899274, 21899721), (21899970, 21900025))


In [16]:
def MakeEnsemblDefault(row):
    llist = row["vcfVariant"].split(":")
    CHR = llist[0]
    Start = int(llist[1])
    End = Start
    RefAlt = llist[2] + "/" + llist[3]
    return [CHR, Start, End, RefAlt]
Jiayao_features = pd.read_csv("unifiedmodel/features.jiayao.csv")
Jiayao_features["#Chr"] = Jiayao_features.apply(lambda row: MakeEnsemblDefault(row)[0], axis=1)
Jiayao_features["#Start"] = Jiayao_features.apply(lambda row: MakeEnsemblDefault(row)[1], axis=1)
Jiayao_features["#End"] = Jiayao_features.apply(lambda row: MakeEnsemblDefault(row)[2], axis=1)
Jiayao_features["#RefAlt"] = Jiayao_features.apply(lambda row: MakeEnsemblDefault(row)[3], axis=1)
print(list(Jiayao_features.columns.values))
Jiayao_features = Jiayao_features[['#Chr', '#Start', '#End', '#RefAlt',
        'KEY', 'familyId', 'vcfVariant', 'inChild', 'effectGene', 
        'effectType', 'gender', 'VIQ', 'NVIQ', 'ExonID', 'GeneExp.amean', 
        'ExonExp.amean', 'GenePrenatalExp.amean', 'ExonPrenatalExp.amean', 
        'GenePostnatalExp.amean', 'ExonPostnatalExp.amean', 'Functional', 
        'DOM.TRUNC.FRAC', 'DOM.TRUNC', 'GERP', 'Rel.exp.amean', 'Rel.exp.gmean', 
        'Rel.exp.prenatal', 'Rel.exp.postnatal', 'Prenatal.Bias', 'phyloP100way', 
        'phastCons100way', 'gender1', 'Age', 'Dosage']]
Jiayao_features.head(2)
Jiayao_features.to_csv("unifiedmodel/features.jiayao.tsv", index=False, sep="\t")

['KEY', 'familyId', 'vcfVariant', 'inChild', 'effectGene', 'effectType', 'gender', 'VIQ', 'NVIQ', 'ExonID', 'GeneExp.amean', 'ExonExp.amean', 'GenePrenatalExp.amean', 'ExonPrenatalExp.amean', 'GenePostnatalExp.amean', 'ExonPostnatalExp.amean', 'Functional', 'DOM.TRUNC.FRAC', 'DOM.TRUNC', 'GERP', 'Rel.exp.amean', 'Rel.exp.gmean', 'Rel.exp.prenatal', 'Rel.exp.postnatal', 'Prenatal.Bias', 'phyloP100way', 'phastCons100way', 'gender1', 'Age', 'Dosage', '#Chr', '#Start', '#End', '#RefAlt']


In [271]:
def isLEJ(Gene, Pos, Ref, Alt, Genes):
    Pos, LenV = int(Pos), len(Ref)-len(Alt)
    gene_obj = Genes[Gene]
    count1, count2, count3 = 0, 0, 0
    for transid, transobj in gene_obj.Transcripts.items():
        if len(transobj.Exons) >= 2:
            count1 += 1
            interval1, interval2 = transobj.LastExonJunction()

            if (Pos > interval1[0] and Pos < interval1[1]):
                count2 += 1
            elif(Pos > interval2[0] and Pos < interval2[1]):
                count3 += 1
    if count2 != 0:
        isle = 'T'
    else:
        isle = "F"
    if count3 != 0:
        islej = 'T'
    else:
        islej = 'F'
    return "{}/{}".format(count2+count3, count1), isle, islej
Jiayao_features = pd.read_csv("unifiedmodel/features.jiayao.csv")
#Jiayao_features.loc[Jiayao_features["effectGene"]=="MLL5", "effectGene"] = "KMT2E" 
for i, row in Jiayao_features.iterrows():
    famid, gene, (Chr, Pos, Ref, Alt) = row["familyId"], row["effectGene"], row["vcfVariant"].split(":")
    nlej, isle,  islej = isLEJ(gene, Pos, Ref, Alt, Genes)
    Jiayao_features.at[i, "NLEJ"] = nlej
    Jiayao_features.at[i, "isLE"] = isle
    Jiayao_features.at[i, "isLEJ"] = islej

TypeError: list indices must be integers or slices, not str

In [48]:
Jiayao_features[(Jiayao_features["isLE"]=="T") | (Jiayao_features["isLEJ"]=="T")]#[["KEY","effectGene","NLEJ","isLE","isLEJ"]]

,KEY,familyId,vcfVariant,inChild,effectGene,effectType,gender,VIQ,NVIQ,ExonID,...,Rel.exp.postnatal,Prenatal.Bias,phyloP100way,phastCons100way,gender1,Age,Dosage,NLEJ,isLE,isLEJ
0,13545-20:49509095,13545,20:49509094:G:GT,pM,ADNP,frame-shift,M,23.0,38.0,41150,...,1.325670,1.423534,2.813101,0.732081,1,61.0,56.455400,4/5,T,F
1,12130-20:49510028,12130,20:49510027:CTT:C,pF,ADNP,frame-shift,F,62.0,55.0,41150,...,1.325670,1.423534,2.813101,0.732081,2,101.0,40.005218,4/5,T,F
8,13447-6:157527665,13447,6:157527664:CTGTT:C,pF,ARID1B,frame-shift,F,32.0,51.0,11552,...,3.028648,0.730105,3.777411,0.875879,2,193.0,46.837116,5/9,T,F
10,13614-15:93563244,13614,15:93563244:C:T,pM,CHD2,nonsense,M,82.0,81.0,183603,...,0.187783,0.469971,0.449107,0.131028,1,113.0,85.151477,2/6,T,F
11,13818-15:93563283,13818,15:93563282:T:TG,pM,CHD2,frame-shift,M,74.0,82.0,183603,...,0.187783,0.469971,0.449107,0.131028,1,119.0,85.076766,2/6,T,F
21,13735-21:41414331,13735,21:41414330:A:AT,pM,DSCAM,frame-shift,M,30.0,27.0,179246,...,2.705670,0.259018,5.494337,0.929677,1,109.0,53.158800,2/2,F,T
27,13256-21:38877892,13256,21:38877891:AGGTCTGTGCTGCTGC:A,pM,DYRK1A,frame-shift,M,70.0,85.0,145379,...,1.140098,0.823883,5.954987,0.947505,1,170.0,41.956807,2/9,F,T
37,13664-1:202698172,13664,1:202698172:G:A,pM,KDM5B,nonsense,M,65.0,63.0,71276,...,2.082489,1.994697,1.740805,0.561544,1,102.0,93.934228,3/5,T,F
41,14299-7:104702707,14299,7:104702706:AC:A,pM,MLL5,frame-shift,M,63.0,75.0,1587,...,0.339825,0.828004,2.327524,0.484296,1,134.0,89.873941,1/12,F,T
43,12969-12:116418554,12969,12:116418553:A:AC,pM,MED13L,splice-site,M,70.0,78.0,79519,...,0.540509,1.665994,1.906456,0.404439,1,146.0,69.302202,1/5,T,F


In [161]:
df = pd.read_excel("/Users/jiayao/Work/BrainDisorders/data/nature13908-s2/Supplementary_Table_2.xlsx")
df = df[['familyId', 'location', 'variant', 'vcfVariant', 'inChild', 'fromParent', 'effectGene', 
         'effectType', 'familyDescription']]
LGD = set(["noStart", "frame-shift", "nonsense", "noEnd", "no-frame-shift-newStop","splice-site"])
MIS = set(["missense"])
SYN = set(["synonymous"])
df_lgd = df[df["effectType"].isin(LGD)]
df_mis = df[df["effectType"].isin(MIS)]
GeneCount = df_lgd.groupby("effectGene")["effectGene"].count()
df_lgd["GeneCount"] = df_lgd.apply(lambda row: GeneCount[row["effectGene"]], axis=1)
df_rec_lgd = df_lgd[df_lgd["GeneCount"]>=2]
df_mis = df_mis[df_mis["familyId"].isin(df_rec_lgd["familyId"].values)]
df_rec_lgd_mis = pd.concat([df_rec_lgd, df_mis], ignore_index=True)

/Users/jiayao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/jiayao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [162]:
def MakeEnsemblDefault(row):
    llist = row["vcfVariant"].split(":")
    CHR = llist[0]
    Start = int(llist[1])
    End = Start
    Ref = llist[2] 
    Alt = llist[3]
    return [CHR, Start, End, Ref, Alt]
df_rec_lgd_mis["#Chr"] = df_rec_lgd_mis.apply(lambda row: MakeEnsemblDefault(row)[0], axis=1)
df_rec_lgd_mis["Start"] = df_rec_lgd_mis.apply(lambda row: MakeEnsemblDefault(row)[1], axis=1)
df_rec_lgd_mis["End"] = df_rec_lgd_mis.apply(lambda row: MakeEnsemblDefault(row)[2], axis=1)
df_rec_lgd_mis["Ref"] = df_rec_lgd_mis.apply(lambda row: MakeEnsemblDefault(row)[3], axis=1)
df_rec_lgd_mis["Alt"] = df_rec_lgd_mis.apply(lambda row: MakeEnsemblDefault(row)[4], axis=1)
print(list(df_rec_lgd_mis.columns.values))
df_rec_lgd_mis = df_rec_lgd_mis[['#Chr', 'Start', 'End', 'Ref', 'Alt']]
df_rec_lgd_mis.head(2)
df_rec_lgd_mis.to_csv("unifiedmodel/iossifov_rec_lgd_mis.tsv", index=False, sep="\t")

['GeneCount', 'effectGene', 'effectType', 'familyDescription', 'familyId', 'fromParent', 'inChild', 'location', 'variant', 'vcfVariant', '#Chr', 'Start', 'End', 'Ref', 'Alt']


In [163]:
anno = pd.read_csv("unifiedmodel/iossifov_rec_lgd_mis.annovar.csv")
anno["vcfVariant"] = anno.apply(lambda row: "{}:{}:{}:{}".format(
    row["Chr"],row["Start"],row["Ref"],row["Alt"]), axis=1)
anno.head(2)
Var2Cadd = dict(zip(anno['vcfVariant'].values, anno["CADD_phred"].values))
for k,v in Var2Cadd.items():
    if v == ".":
        Var2Cadd[k] = 0

In [164]:
#Select subset of mis and check lgd+mis
df_mis["CADD_phred"] = df_mis.apply(lambda row: float(Var2Cadd.get(row["vcfVariant"], 0)), axis=1)
df_mis = df_mis[(df_mis["effectType"]=="missense") & (df_mis["CADD_phred"]==0)]
#display(df_mis)

In [165]:
df_rec_lgd_mis = pd.concat([df_rec_lgd, df_mis], ignore_index=True)

/Users/jiayao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [166]:
df_rec_lgd_mis = df_rec_lgd_mis.sort_values(by="familyId")
FamCount = df_rec_lgd_mis.groupby("familyId")["familyId"].count()
df_rec_lgd_mis["FamCount"] = df_rec_lgd_mis.apply(lambda row: FamCount[row["familyId"]], axis=1)
ha = df_rec_lgd_mis[df_rec_lgd_mis["FamCount"]>=2]
print(list(set(ha["familyId"].values)))
out = open("unifiedmodel/cadd.0.list", 'wt')
for i in list(set(ha["familyId"].values)):
    out.write(str(i)+"\n")

[12600, 14376, 12810, 12752]


In [154]:
#display(df_rec_lgd_mis[df_rec_lgd_mis["FamCount"]>=2])
ha[ha["effectGene"]=="KMT2E"]
ha[ha["familyId"]==14299]

,CADD_phred,GeneCount,effectGene,effectType,familyDescription,familyId,fromParent,inChild,location,variant,vcfVariant,FamCount


In [221]:
Genes = ["ASXL3","CHD2","CHD8","DSCAM","DYRK1A","FOXP1","HIVEP2","SCN2A"]
Genes = ["ASXL3","CHD2","CHD8","DSCAM","DYRK1A","FOXP1","HIVEP2","SCN2A","ADNP","CHAMP1","CSNK2A1","GRIN2B",
        "HNRNPH2","MED13L","PACS1","PPP2R5D","SETBP1","STXBP1","SYNGAP1"]
gtf = "/Users/jiayao/Work/BrainDisorders/data/gencode.v19.annotation.gtf"
out = open("unifiedmodel/VIPgenes.gencode.v19.gtf", 'wt')
hand = open(gtf, 'rt')
for l in hand:
    if l.startswith("#"):
        out.write(l)
        continue
    llist = l.strip().split("\t")
    info = gtf_info_parser(llist[8]) 
    if info["gene_name"] in Genes:
        out.write(l)

In [254]:
Gene2Trans = {}
hand = open("unifiedmodel/VIPgenes.gencode.v19.gtf", 'rt')
for l in hand:
    if l.startswith("#"):
        continue
    llist = l.strip().split("\t")
    info = gtf_info_parser(llist[8]) 
    CHR = llist[0].lstrip("chr")
    strand = llist[6]
    start = (llist[3])
    end = (llist[4])
    if llist[2] == "transcript" and info["transcript_type"] == "protein_coding":
        Gene2Trans[info["gene_name"]] = info["transcript_id"]
Gene2Trans["ASXL3"]="ENST00000269197.5"
Gene2Trans["CHD8"]="ENST00000399982.2"
Gene2Trans["DYRK1A"]="ENST00000339659.4"
Gene2Trans["CHD2"]="ENST00000394196.4"
Gene2Trans["SCN2A"]="ENST00000357398.3"
Gene2Trans["FOXP1"]="ENST00000318789.4"
Gene2Trans["HIVEP2"]="ENST00000367603.2"
Gene2Trans["PPP2R5D"]="ENST00000485511.1"
Gene2Trans["HIVEP2"]="ENST00000367603.2"
Gene2Trans["SYNGAP1"]="ENST00000418600.2"
Gene2Trans["STXBP1"]="ENST00000373302.3"
Gene2Trans["PACS1"]="ENST00000320580.4"
Gene2Trans["MED13L"]="ENST00000281928.3"
Gene2Trans["CHAMP1"]="ENST00000361283.1"
Gene2Trans["SETBP1"]="ENST00000282030.5"
Gene2Trans["CSNK2A1"]="ENST00000217244.3"
Gene2Trans["ADNP"]="ENST00000396029.3"
Gene2Trans["DSCAM"]="ENST00000400454.1"
Gene2Trans["HNRNPH2"]="ENST00000400454.1"
print (Gene2Trans)

{'SCN2A': 'ENST00000357398.3', 'FOXP1': 'ENST00000318789.4', 'SYNGAP1': 'ENST00000418600.2', 'PPP2R5D': 'ENST00000485511.1', 'HIVEP2': 'ENST00000367603.2', 'STXBP1': 'ENST00000373302.3', 'PACS1': 'ENST00000320580.4', 'GRIN2B': 'ENST00000609686.1', 'MED13L': 'ENST00000281928.3', 'CHAMP1': 'ENST00000361283.1', 'CHD8': 'ENST00000399982.2', 'CHD2': 'ENST00000394196.4', 'ASXL3': 'ENST00000269197.5', 'SETBP1': 'ENST00000282030.5', 'CSNK2A1': 'ENST00000217244.3', 'ADNP': 'ENST00000396029.3', 'DYRK1A': 'ENST00000339659.4', 'DSCAM': 'ENST00000400454.1', 'HNRNPH2': 'ENST00000400454.1'}


In [283]:
# VIP phenotype
DIR = "/Users/jiayao/Work/BrainDisorders/VIP_phenotype/Simons_VIP_Phase_2_Single_Gene_Dataset_v5.0"
#Genes = ["ASXL3","CHD2","CHD8","DSCAM","DYRK1A","FOXP1","HIVEP2","SCN2A"]
#Genes = ["ASXL3","DYRK1A","HIVEP2","SCN2A"]
dfs1 = []
dfs2 = []
for gene in Genes:
    try:
        df1 = pd.read_csv("/".join([DIR,gene,"vineland_ii.csv"]))
        dfs1.append(df1)
    except:
        pass
    try:
        df2 = pd.read_csv("/".join([DIR,gene,"lab_results.csv"]))
        dfs2.append(df2)
    except:
        pass
VABS_DF = pd.concat(dfs1)
LGD_DF = pd.concat(dfs2)

In [284]:
LGD_DF = LGD_DF[["sfari_id", "relationship_to_iip", "genetic_status","lab_results_single_hgvs"]]
LGD_DF = LGD_DF[LGD_DF["relationship_to_iip"]=="Initially identified proband"]
VABS_DF = VABS_DF[VABS_DF["relationship_to_iip"]=="Initially identified proband"]
#LGD_DF = LGD_DF.dropna()
LGD_DF.head(2)

,sfari_id,relationship_to_iip,genetic_status,lab_results_single_hgvs
0,15710-x1,Initially identified proband,ASXL3,3964C>T
3,15715-x1,Initially identified proband,ASXL3,NaN


In [286]:
LGD_DF.shape[0]
len(set(LGD_DF["sfari_id"].values))

321

In [287]:
15+26+7+5+10+12+30+10+11+14+20+28+83+8+32+23

334

In [274]:
VABS_DF = VABS_DF[["sfari_id","sex","genetic_status","inheritance_status", "composite_standard_score"]]
VABS_DF = VABS_DF.reset_index(drop=True)
VABS_DF.head(2)

,sfari_id,sex,genetic_status,inheritance_status,composite_standard_score
0,15710-x1,male,ASXL3,de-novo,75.0
1,16159-x1,female,ASXL3,de-novo,37.0


In [275]:
VABS_DF = VABS_DF.dropna()
VABS_DF.head()

,sfari_id,sex,genetic_status,inheritance_status,composite_standard_score
0,15710-x1,male,ASXL3,de-novo,75.0
1,16159-x1,female,ASXL3,de-novo,37.0
2,16171-x1,female,ASXL3,de-novo,32.0
3,16179-x1,female,ASXL3,de-novo,57.0
4,16185-x1,male,ASXL3,de-novo,68.0


In [276]:
def addHGVS(row, Gene2Trans):
    Trans = Gene2Trans[row["genetic_status"]]
    hgvs = row["lab_results_single_hgvs"]
    hgvs = "c.{}".format(hgvs) if not (hgvs.startswith("c.")) else hgvs
    return "{}:{}".format(Trans, hgvs)
LGD_DF["HGVS"] = LGD_DF.apply(lambda row:addHGVS(row, Gene2Trans), axis=1)

In [277]:
LGD_DF.to_csv("unifiedmodel/VIP.Genes.HGVS.tsv", sep="\t", index=False)

In [278]:
Transcripts = [x for x in Gene2Trans.values()]
Transcripts

['ENST00000357398.3',
 'ENST00000318789.4',
 'ENST00000418600.2',
 'ENST00000485511.1',
 'ENST00000367603.2',
 'ENST00000373302.3',
 'ENST00000320580.4',
 'ENST00000609686.1',
 'ENST00000281928.3',
 'ENST00000361283.1',
 'ENST00000399982.2',
 'ENST00000394196.4',
 'ENST00000269197.5',
 'ENST00000282030.5',
 'ENST00000217244.3',
 'ENST00000396029.3',
 'ENST00000339659.4',
 'ENST00000400454.1',
 'ENST00000400454.1']

In [279]:
VEP = pd.read_csv("unifiedmodel/VIP.hgvs.vep.txt", delimiter="\t")
Transcripts = [x for x in Gene2Trans.values()]
LGDs = ['frameshift_variant', 'missense_variant,splice_region_variant', 
        'stop_gained', 'splice_donor_variant']
VEP = VEP[(VEP["Feature"].isin(Transcripts)) & (VEP["Consequence"].isin(LGDs))]
VEP = VEP.drop_duplicates(subset="#Uploaded_variation", keep='first')
VEP.set_index("#Uploaded_variation", inplace=True)
VEP.to_csv("unifiedmodel/VIP.VEP.RecGenes.LGD.txt", sep="\t")
print(VEP.shape)

(115, 47)


In [280]:
def serachVEP(row, vip_hgvs, Gene2Trans, VEP):
    transcript = Gene2Trans[row["genetic_status"]]
    vip_hgvs = "c.{}".format(vip_hgvs) if not (vip_hgvs.startswith("c.")) else vip_hgvs
    key = "{}:{}".format(transcript, vip_hgvs)
    location = VEP.loc[key, "Location"]
    return location
    
for i, row in VABS_DF.iterrows():
    spid, gene = row["sfari_id"], row["genetic_status"]
    vip_hgvs = LGD_DF[(LGD_DF["sfari_id"]==spid) & ((LGD_DF["genetic_status"]==gene))]
    if vip_hgvs.shape[0] == 1:
        vip_hgvs = vip_hgvs["lab_results_single_hgvs"].values[0]
        try:
            VABS_DF.at[i, "location"] = serachVEP(row, vip_hgvs, Gene2Trans, VEP)
        except KeyError:
            #print(vip_hgvs)
            VABS_DF.at[i, "location"] = np.nan
    else:
        VABS_DF.at[i, "location"] = np.nan
    VABS_DF.at[i, "sex"] = row["sex"][0].upper()

In [281]:
print(VABS_DF.shape[0])
VABS_DF = VABS_DF.dropna()
print(VABS_DF.shape[0])

356
123


In [270]:
#VABS_DF.to_csv("unifiedmodel/vip.recgenes.withlocation.csv", index=False)

In [282]:
VABS_DF.head(2)

,sfari_id,sex,genetic_status,inheritance_status,composite_standard_score,location
0,15710-x1,M,ASXL3,de-novo,75.0,18:31323776-31323776
1,16159-x1,F,ASXL3,de-novo,37.0,18:31319345-31319349


In [206]:
Jiayao_features = pd.read_csv("unifiedmodel/features.jiayao.csv")
#Jiayao_features.loc[Jiayao_features["effectGene"]=="MLL5", "effectGene"] = "KMT2E" 
gtex = pd.read_csv("../data/GTEx.RecGenes.exon.csv")
Exons = pd.read_csv("../data/gencode.v19.genes.v7.patched_contigs.exons.txt", delimiter="\t")

In [214]:
Jiayao_features.head(2)

,KEY,familyId,vcfVariant,inChild,effectGene,effectType,gender,VIQ,NVIQ,ExonID,...,Rel.exp.amean,Rel.exp.gmean,Rel.exp.prenatal,Rel.exp.postnatal,Prenatal.Bias,phyloP100way,phastCons100way,gender1,Age,Dosage
0,13545-20:49509095,13545,20:49509094:G:GT,pM,ADNP,frame-shift,M,23.0,38.0,41150,...,1.248356,1.29008,1.215442,1.32567,1.423534,2.813101,0.732081,1,61.0,56.455400
1,12130-20:49510028,12130,20:49510027:CTT:C,pF,ADNP,frame-shift,F,62.0,55.0,41150,...,1.248356,1.29008,1.215442,1.32567,1.423534,2.813101,0.732081,2,101.0,40.005218


In [215]:
print(len(set(Jiayao_features["effectGene"].values)))

27


In [207]:
Exons.head(2)

,exon_id,chr,start_pos,end_pos,strand
0,ENSG00000223972.4_1,1,11869,12227,+
1,ENSG00000223972.4_2,1,12595,12721,+


In [208]:
gtex.head(2)

,Description,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,...,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O,Name
0,POGZ,4.947368,0.0,14.092105,7.355263,0.486842,5.828947,10.421053,9.776316,8.513158,...,12.078947,11.460526,4.407895,10.671053,21.223684,7.881579,6.697368,9.157895,8.039474,ENSG00000143442.17_1
1,POGZ,4.000000,1.0,7.763158,25.171053,7.000000,17.552631,19.842106,14.578947,20.171053,...,25.381578,13.302632,12.539474,20.657894,34.197370,4.105263,9.460526,1.000000,7.000000,ENSG00000143442.17_2


In [211]:
len(set(gtex["Description"].values))

27

In [219]:
gtex = pd.read_csv("../data/GTEx.RecGenes.exon.csv")
RecGenes = set(gtex["Description"].values)
out = csv.writer(open("../data/GTEx.RecGenes.gene.rc.csv", 'wt'))
for l in open("../data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct"):
    if l.startswith("Name"):
        header = l.strip().split("\t")
        out.writerow(header)
    else:
        llist = l.strip().split("\t")
        if llist[1] in RecGenes:
            out.writerow(llist)

In [ ]:
csv.field_size_limit(sys.maxsize)
reader = csv.reader(open("../data/GTEx/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_tpm.txt", 'rt'), delimiter="\t")
writer = csv.writer(open("../data/GTEx/GTEx_180Indiv_transcript_tpm.txt", 'wt'), delimiter="\t")
header = next(reader)
indices = [0,1]
new_header = header[:2]
for i,xx in enumerate(header[2:]):
    doner = xx.split("-")[1]
    if doner in Doners:
        indices.append(i+2)
        new_header.append(xx)
writer.writerow(new_header)
for row in reader:
    new_row = []
    for i in indices:
        new_row.append(row[i])
    writer.writerow(new_row)

In [ ]:
csv.field_size_limit(sys.maxsize)
reader = csv.reader(open("../data/GTEx/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct", 'rt'), delimiter="\t")
writer = csv.writer(open("../data/GTEx/GTEx_180Indiv_gene_tpm.txt", 'wt'), delimiter="\t")
header = next(reader)
indices = [0,1]
new_header = header[:2]
for i,xx in enumerate(header[2:]):
    doner = xx.split("-")[1]
    if doner in Doners:
        indices.append(i+2)
        new_header.append(xx)
writer.writerow(new_header)
for row in reader:
    new_row = []
    for i in indices:
        new_row.append(row[i])
    writer.writerow(new_row)

In [290]:
# Select used gencode gtf data
SSC_All_LGD = pd.read_csv("unifiedmodel/SSC.LGD.features.csv")
gtex_lgd_genes = set(SSC_All_LGD["effectGene"].values)
gtf = "/Users/jiayao/Work/BrainDisorders/data/gencode.v19.annotation.gtf"
out = open("../data/GTEx/SSC.LGDgenes.gencode.v19.gtf", 'wt')
hand = open(gtf, 'rt')
for l in hand:
    if l.startswith("#"):
        out.write(l)
        continue
    llist = l.strip().split("\t")
    info = gtf_info_parser(llist[8])
    if llist[2] == "gene" and info["gene_name"] in gtex_lgd_genes:
        out.write(l)
    elif llist[2] == 'transcript' and info["gene_name"] in gtex_lgd_genes and info["transcript_type"]=="protein_coding":
        out.write(l)
    elif llist[2] == 'exon' and info["gene_name"] in gtex_lgd_genes and info["transcript_type"]=="protein_coding":  
        out.write(l)
    elif llist[2] == 'CDS' and info["gene_name"] in gtex_lgd_genes and info["transcript_type"]=="protein_coding":  
        out.write(l)